In [4]:
import psycopg2
import pandas as pd

# === Config ===
DB_NAME = "camping"
DB_USER = "arrowboy"
CSV_PATH = "data/angeles.csv"  # <-- UPDATE THIS

# === Load CSV ===
df = pd.read_csv(CSV_PATH)

# Replace NaNs with None (so psycopg2 sends NULL to SQL)
df = df.where(pd.notnull(df), None)

try:
    conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER)
    cur = conn.cursor()

    for _, row in df.iterrows():
        # === Insert into campsites ===
        cur.execute("""
            INSERT INTO campsites (
                name, latitude, longitude, address, managing_unit,
                reservation_type, fs_usda_url, reservation_url,
                contact_name, contact_phone, seasons_of_use,
                num_sites, fee, overview
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            RETURNING id;
        """, (
            row['site_name'],
            row['latitude'],
            row['longitude'],
            row['directions'],
            row['info_center'],
            None,                     # reservation_type
            row['park_url'],
            None,                     # reservation_url
            row['info_center'],       # contact_name from info_center
            row['contact_info'],
            row['season_of_use'],
            None,                     # num_sites
            row['fee_info'],
            row['overview']
        ))

        campsite_id = cur.fetchone()[0]  # Get the inserted row's ID

        # === Insert into amenities ===
        cur.execute("""
            INSERT INTO amenities (
                campsite_id, water_activities, boating,
                restrooms, water, amenities
            ) VALUES (%s, %s, %s, %s, %s, %s);
        """, (
            campsite_id,
            row['body_of_water'],  # assumed boolean True/False
            None,                  # no boating info
            row['restrooms'],
            row['water'],
            row['amenities']
        ))

    conn.commit()
    print("✅ Data inserted successfully.")

except Exception as e:
    print("❌ Error inserting data:", e)
    if conn:
        conn.rollback()

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()


❌ Error inserting data: column "water_activities" of relation "amenities" does not exist
LINE 3:                 campsite_id, water_activities, boating,
                                     ^

